In [1]:
import os
import sys
import requests
import json
from typing import Set

# --- Environment Variable for API Keys ---
# This is the environment variable that needs to be set in your shell before launching Jupyter.
URLHAUS_AUTH_KEY_ENV_VAR = "URLHAUS_AUTH_KEY"
SIEM_API_KEY_ENV_VAR = "SIEM_API_KEY" # Placeholder for a SIEM API key

# --- Global lists to be populated with threat intelligence ---
# These lists are initially empty and will be filled in a subsequent cell.
KNOWN_BAD_DOMAINS: Set[str] = set()
KNOWN_BAD_IPS: Set[str] = set()
SUSPICIOUS_KEYWORDS: Set[str] = {
    'urgent', 'password', 'invoice', 'payment', 'suspended', 
    'verify', 'action required', 'click here'
}

# --- Standard Incident Data Structure (for reference) ---
# This function is used to create a consistent data structure for alerts.
def create_incident_alert(alert_id: str, source_ip: str, sender_domain: str, sender_email: str, subject: str, body: str) -> dict:
    """Creates a standardized incident alert dictionary for testing."""
    return {
        "alert_id": alert_id,
        "source_ip": source_ip,
        "sender_domain": sender_domain,
        "sender_email": sender_email,
        "subject": subject,
        "body": body,
        "triage_score": 0,
        "triage_summary": []
    }

# --- Test Alert ---
# This is a sample alert for testing the playbook's functionality.
# In a real-world scenario, this would be retrieved from a SIEM.
TEST_ALERT = {
    "alert_id": "PHISH-TEST-01",
    "source_ip": "203.0.113.25", 
    "sender_domain": "malicious-phish-site.net", 
    "sender_email": "support@malicious-phish-site.net",
    "subject": "URGENT ACTION REQUIRED: Account Security Alert",
    "body": "Dear user, we have detected suspicious activity on your account. Please click here to verify your identity and prevent account suspension."
}

In [2]:
# --- ACTIONABLE: This cell contains API functions for threat intelligence. ---

def fetch_recent_urlhaus_domains() -> Set[str]:
    """
    Fetches the latest malicious domains from the URLhaus CSV recent feed.
    
    This function uses the more reliable CSV API endpoint to retrieve and parse
    the list of recently submitted malicious URLs.
    
    Returns:
        Set[str]: A set of unique domains from the recent URLhaus submissions.
    """
    url = "https://urlhaus.abuse.ch/downloads/csv_recent/"
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        lines = response.text.splitlines()
        domains = set()
        
        for line in lines:
            # Skip comments and header lines, which start with '#'
            if line.strip().startswith("#"):
                continue
            
            # The URL is the third column in the CSV (index 2)
            parts = line.split(',')
            if len(parts) > 2:
                full_url = parts[2].strip().replace('"', '')
                try:
                    domain = full_url.split('/')[2].split(':')[0]
                    domains.add(domain)
                except IndexError:
                    # Skip malformed URLs
                    continue
        
        if len(domains) > 0:
            print(f"Successfully fetched {len(domains)} unique domains from URLhaus recent feed.")
        else:
            print("Successfully fetched URLhaus feed, but no domains were found. The feed may be empty.")
            
        return domains
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URLhaus recent feed: {e}")
        return set()

def query_urlhaus_single(url: str) -> dict:
    """
    Queries the URLhaus API for a single URL using an authentication key.
    
    This function demonstrates secure API integration using an environment variable
    and is useful for enriching data for a specific indicator.
    
    Args:
        url (str): The URL to query.
    
    Returns:
        dict: The JSON response from the API, or None if the query fails.
    """
    auth_key = os.getenv(URLHAUS_AUTH_KEY_ENV_VAR)
    if not auth_key:
        print(f"Error: Environment variable '{URLHAUS_AUTH_KEY_ENV_VAR}' is not set.")
        return None

    data = {'url': url}
    headers = {"Auth-Key": auth_key}
    
    try:
        response = requests.post('https://urlhaus-api.abuse.ch/v1/url/', data, headers=headers, timeout=15)
        response.raise_for_status()
        json_response = response.json()
        
        return json_response
    except requests.exceptions.RequestException as e:
        print(f"Error querying URLhaus for {url}: {e}")
        return None

In [3]:
# --- ACTIONABLE: Populate the global threat intelligence lists. ---

print("Populating KNOWN_BAD_DOMAINS with recent URLhaus submissions...")
recent_domains = fetch_recent_urlhaus_domains()
KNOWN_BAD_DOMAINS.update(recent_domains)

# --- Placeholder for other threat intelligence feeds ---
# You can add other sources here to populate KNOWN_BAD_IPS or other lists.
# For example:
# KNOWN_BAD_IPS.update(fetch_threat_intel_list("https://path-to-a-malicious-ip-feed.txt"))

# Print a sample of the populated lists to verify.
print("\n--- Playbook Setup Complete ---")
print(f"Total domains in KNOWN_BAD_DOMAINS: {len(KNOWN_BAD_DOMAINS)}")
print(f"Sample domains: {list(KNOWN_BAD_DOMAINS)[:5]}")
print(f"Total keywords in SUSPICIOUS_KEYWORDS: {len(SUSPICIOUS_KEYWORDS)}")
print("Test alert data is ready in the variable 'TEST_ALERT'.")

Populating KNOWN_BAD_DOMAINS with recent URLhaus submissions...
Successfully fetched 7037 unique domains from URLhaus recent feed.

--- Playbook Setup Complete ---
Total domains in KNOWN_BAD_DOMAINS: 7037
Sample domains: ['107.189.27.205', '123.190.138.216', '223.12.185.219', '117.235.110.55', '123.9.245.68']
Total keywords in SUSPICIOUS_KEYWORDS: 8
Test alert data is ready in the variable 'TEST_ALERT'.


In [4]:
# --- ACTIONABLE: This is the core logic of the playbook. ---

def initial_phishing_triage(incident: dict) -> dict:
    """
    Performs initial triage on a phishing-related incident using known IoCs.
    
    This function analyzes an alert to check for indicators of a phishing attack
    and assigns a triage score and summary. The score helps determine the incident's
    severity and priority.
    
    Args:
        incident (dict): The incident alert to analyze.
        
    Returns:
        dict: The updated incident dictionary with a triage score and summary.
    """
    triage_score = 0
    triage_summary = []
    
    # Check against Known Bad Domains (MITRE ATT&CK T1566.001)
    if incident['sender_domain'] in KNOWN_BAD_DOMAINS:
        triage_score += 50
        triage_summary.append("MATCH: Sender domain is a known bad domain from URLhaus.")
        
    # Check against Known Bad IPs (MITRE ATT&CK T1071.001)
    # This check is currently using a mock list, but would be populated in a real playbook.
    if incident['source_ip'] in KNOWN_BAD_IPS:
        triage_score += 30
        triage_summary.append("MATCH: Source IP is a known malicious IP address.")
        
    # Check for suspicious keywords in subject and body
    combined_text = incident['subject'] + " " + incident['body']
    found_keywords = [kw for kw in SUSPICIOUS_KEYWORDS if kw in combined_text.lower()]
    
    if found_keywords:
        triage_score += 10 * len(found_keywords)
        triage_summary.append(f"MATCH: Found suspicious keywords: {', '.join(found_keywords)}.")
        
    # Update the incident dictionary with the triage results
    incident['triage_score'] = triage_score
    incident['triage_summary'] = triage_summary
    
    return incident

In [5]:
# --- ACTIONABLE: This cell executes the triage function on a specific alert. ---

print("--- Initiating Triage for Test Alert ---")
print(f"Alert ID: {TEST_ALERT['alert_id']}")
print(f"Sender Domain: {TEST_ALERT['sender_domain']}")

# Execute the triage logic
triage_results = initial_phishing_triage(TEST_ALERT)

# Define severity thresholds based on the final score
if triage_results['triage_score'] >= 50:
    severity = "High"
    mitre_techniques = "T1566.001 (Spearphishing Link)"
elif triage_results['triage_score'] >= 30:
    severity = "Medium"
    mitre_techniques = "T1071.001 (Application Layer Protocol)"
else:
    severity = "Low"
    mitre_techniques = "N/A"
    
print("\n--- Triage Summary ---")
print(f"Final Triage Score: {triage_results['triage_score']}")
print(f"Severity Level: {severity}")
print(f"MITRE ATT&CK Mapping: {mitre_techniques}")
print("Detailed Findings:")
for finding in triage_results['triage_summary']:
    print(f"- {finding}")

if severity == "High":
    print("\n[ACTION REQUIRED]: This is a high-priority incident. It requires immediate analyst intervention and containment.")
else:
    print("\n[FOR REVIEW]: This is a low-to-medium priority incident. Review the findings for potential false positives or further investigation.")

--- Initiating Triage for Test Alert ---
Alert ID: PHISH-TEST-01
Sender Domain: malicious-phish-site.net

--- Triage Summary ---
Final Triage Score: 40
Severity Level: Medium
MITRE ATT&CK Mapping: T1071.001 (Application Layer Protocol)
Detailed Findings:
- MATCH: Found suspicious keywords: action required, urgent, click here, verify.

[FOR REVIEW]: This is a low-to-medium priority incident. Review the findings for potential false positives or further investigation.


In [6]:
# --- ACTIONABLE: This cell performs enrichment on a specific indicator. ---

print("--- Starting Enrichment with URLhaus Single-URL Lookup ---")

# Access the sender's domain from the previously triaged alert
domain_to_check = TEST_ALERT['sender_domain']
print(f"Querying URLhaus for the domain: {domain_to_check}")

# Execute the single-URL lookup function
urlhaus_response = query_urlhaus_single(domain_to_check)

if urlhaus_response:
    print("\nURLhaus API Response:")
    print(json.dumps(urlhaus_response, indent=4))
    
    # Check the API response to see if the domain is flagged
    if urlhaus_response.get('query_status') == 'ok':
        print("\n[CONFIRMED MALICIOUS]: The URLhaus lookup confirmed this domain is malicious.")
        # An analyst would now proceed with high-priority containment actions.
        # You could also update the alert score here if this was a separate step.
    elif urlhaus_response.get('query_status') == 'no_results':
        print("\n[CLEAN]: The URLhaus lookup did not find any malicious records for this domain.")
    else:
        print("\n[UNKNOWN]: The URLhaus lookup returned an unexpected status. Further manual investigation is required.")
else:
    print("\n[ENRICHMENT FAILED]: Unable to get a response from the URLhaus API. Check your API key and connection.")

--- Starting Enrichment with URLhaus Single-URL Lookup ---
Querying URLhaus for the domain: malicious-phish-site.net
Error: Environment variable 'URLHAUS_AUTH_KEY' is not set.

[ENRICHMENT FAILED]: Unable to get a response from the URLhaus API. Check your API key and connection.


In [7]:
# --- ACTIONABLE: This cell outlines the recommended next steps based on the playbook's findings. ---

# Assume the previous cells have been executed and the 'triage_results' dictionary is populated.
final_score = triage_results.get('triage_score', 0)
sender_domain = triage_results.get('sender_domain', 'N/A')

print("--- Incident Action Plan ---")

if final_score >= 50:
    print("Based on a high triage score, this incident is confirmed to be a high-severity phishing attack.")
    print("Recommended Immediate Actions:")
    print("1. Containment: Isolate the affected user's device and block the malicious domain.")
    print("2. Eradication: Force a password reset for the affected user and check for any unauthorized access.")
    print("3. Communication: Notify the user and other relevant stakeholders (e.g., security leadership, legal).")
    print("4. Enrichment: Check for other users who may have received the same email or visited the malicious domain.")
    print(f"\n[NEXT AUTOMATION STEP]: You can now run a script to automatically block '{sender_domain}' at your firewall or proxy.")

elif final_score >= 30:
    print("Based on a medium triage score, this incident requires further investigation.")
    print("Recommended Next Steps:")
    print("1. User Interview: Contact the user to verify if the email is legitimate or suspicious.")
    print("2. Log Analysis: Review email gateway and proxy logs to see if other users received this email.")
    print("3. Enrichment: Perform additional lookups on the sender domain and IP using other threat intelligence sources.")
    print("4. Monitoring: Add the domain to your watch list for future detections.")

else:
    print("Based on a low triage score, this incident is likely a low-risk event or a false positive.")
    print("Recommended Next Steps:")
    print("1. Close the Alert: Document the findings and close the ticket with a 'false positive' or 'no action required' status.")
    print("2. Tune Detections: Review the detection logic to prevent similar low-risk alerts in the future.")

--- Incident Action Plan ---
Based on a medium triage score, this incident requires further investigation.
Recommended Next Steps:
1. User Interview: Contact the user to verify if the email is legitimate or suspicious.
2. Log Analysis: Review email gateway and proxy logs to see if other users received this email.
3. Enrichment: Perform additional lookups on the sender domain and IP using other threat intelligence sources.
4. Monitoring: Add the domain to your watch list for future detections.


In [8]:
# --- ACTIONABLE: This cell executes a containment action. ---

def block_domain_at_proxy(domain: str) -> bool:
    """
    Simulates blocking a domain at a network proxy or firewall.
    
    This is a placeholder function. In a real playbook, you would replace
    this code with API calls to your specific security tools.
    
    Args:
        domain (str): The malicious domain to block.
        
    Returns:
        bool: True if the action was successful, False otherwise.
    """
    # Placeholder for API call to your firewall, proxy, or DNS filter
    print(f"Executing API call to block domain: {domain}...")
    try:
        # Example API call (replace with your actual integration)
        # response = requests.post("https://my-proxy-api.com/block_list", json={"domain": domain})
        # response.raise_for_status()
        print(f"Successfully sent block request for {domain} to the security control.")
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error blocking domain {domain}: {e}")
        return False

# Check the triage results to decide if containment is necessary
if triage_results.get('triage_score', 0) >= 50:
    sender_domain = triage_results.get('sender_domain', 'unknown')
    print("\n--- Initiating Automated Containment ---")
    print(f"High-severity incident detected. Attempting to block malicious domain: {sender_domain}")
    
    success = block_domain_at_proxy(sender_domain)
    
    if success:
        print(f"\n[CONTAINMENT SUCCESSFUL]: The domain '{sender_domain}' has been submitted for blocking. You can now proceed with eradication and recovery steps.")
    else:
        print(f"\n[CONTAINMENT FAILED]: The automated block failed. Proceed with manual containment immediately.")
else:
    print("\n[NO ACTION REQUIRED]: The incident is not of high enough severity to warrant an automated containment action.")


[NO ACTION REQUIRED]: The incident is not of high enough severity to warrant an automated containment action.
